<a href="https://colab.research.google.com/github/ThomasGVoss/LearningFactory/blob/main/Lab_Data_Prep_with_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import zipfile                                    # For unzipping

# ensure graphs are displayed correctly inline in this notebook
%matplotlib inline

In [31]:
!wget https://raw.githubusercontent.com/ThomasGVoss/LearningFactory/main/apjournal.csv
!wget https://raw.githubusercontent.com/ThomasGVoss/LearningFactory/main/kundenauftrag.csv
!wget https://raw.githubusercontent.com/ThomasGVoss/LearningFactory/main/produktionsauftrag.csv


--2022-09-03 16:40:10--  https://raw.githubusercontent.com/ThomasGVoss/LearningFactory/main/apjournal.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 297142 (290K) [text/plain]
Saving to: ‘apjournal.csv’

apjournal.csv       100%[===================>] 290.18K  --.-KB/s    in 0.03s   

2022-09-03 16:40:10 (8.79 MB/s) - ‘apjournal.csv’ saved [297142/297142]

--2022-09-03 16:40:10--  https://raw.githubusercontent.com/ThomasGVoss/LearningFactory/main/kundenauftrag.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277956 (271K) [

In [32]:
col = ['ProcessID','RoundId','Workstation','Null','Start','End']
data = pd.read_csv('/content/apjournal.csv', header=None, names=col, index_col=0 , sep=',',on_bad_lines='skip')
pd.set_option('display.max_columns', 500)   # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20) # Keep the output on one page
data = data.drop(columns='Null')
#last output of a cell is automatically displayed in this case the pandas DataFrame
data

,ProcessID,RoundId,Workstation,Start,End
1,76,31,1,2018-01-10 21:00:50,2018-01-10 21:05:09
2,76,31,1,2018-01-10 21:05:16,2018-01-10 21:05:21
3,77,31,1,2018-01-10 21:13:29,2018-01-10 21:13:56
4,78,31,1,2018-01-10 21:20:21,2018-01-12 15:59:51
5,78,31,2,2018-01-10 21:20:33,2018-01-10 21:20:35
...,...,...,...,...,...
4217,1762,216,2,2022-09-02 20:05:56,2022-09-02 20:07:07
4218,1759,216,5,2022-09-02 20:06:51,2022-09-02 20:07:20
4219,1762,216,3,2022-09-02 20:07:09,2022-09-02 20:07:31
4220,1762,216,4,2022-09-02 20:07:46,2022-09-02 20:08:12


## Let's take a look into the round we played and talk about the data. 



In [33]:
data = data.loc[data['RoundId'] == 216]

At a high level, we can see:

...

In [34]:
data['Start'] = pd.to_datetime(data['Start'])
data['End'] = pd.to_datetime(data['End'])
data.dtypes


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


ProcessID               int64
RoundId                 int64
Workstation             int64
Start          datetime64[ns]
End            datetime64[ns]
dtype: object

In [24]:
# Select a column
ws = data['Workstation']
ws

4125    1
4126    2
4127    1
4128    1
4129    3
       ..
4217    2
4218    5
4219    3
4220    4
4221    5
Name: Workstation, Length: 97, dtype: int64

In [25]:
data.sort_values(by="Start")

,ProcessID,RoundId,Workstation,Start,End,Seconds
4125,1745,216,1,2022-09-02 19:29:26,2022-09-02 19:30:27,61.0
4126,1745,216,2,2022-09-02 19:30:29,2022-09-02 19:32:32,123.0
4127,1746,216,1,2022-09-02 19:30:36,2022-09-02 19:32:01,85.0
4128,1747,216,1,2022-09-02 19:32:23,2022-09-02 19:33:21,58.0
4129,1745,216,3,2022-09-02 19:32:35,2022-09-02 19:34:29,114.0
...,...,...,...,...,...,...
4217,1762,216,2,2022-09-02 20:05:56,2022-09-02 20:07:07,71.0
4218,1759,216,5,2022-09-02 20:06:51,2022-09-02 20:07:20,29.0
4219,1762,216,3,2022-09-02 20:07:09,2022-09-02 20:07:31,22.0
4220,1762,216,4,2022-09-02 20:07:46,2022-09-02 20:08:12,26.0


In [26]:
# Grouping
data.groupby('Workstation').size()

Workstation
1    18
2    17
3    18
4    25
5    19
dtype: int64

In [27]:
#Generate the duration based on the end and the start date
data['Duration'] = data['End'] - data['Start']

# Series.dt - Accessor object for datetimelike properties of the Series values.
data['Seconds'] = data['Duration'].dt.total_seconds()

#drop the duration col 
data = data.drop(columns=['Duration'])

## Exploration
Let's start exploring the data. First, let's understand how the features are distributed.

In [28]:
data.describe()

,ProcessID,RoundId,Workstation,Seconds
count,97.000000,97.0,97.000000,96.000000
mean,1752.680412,216.0,3.103093,75.302083
std,5.312223,0.0,1.403007,73.804738
min,1745.000000,216.0,1.000000,1.000000
25%,1748.000000,216.0,2.000000,30.500000
50%,1753.000000,216.0,3.000000,62.000000
75%,1757.000000,216.0,4.000000,94.000000
max,1762.000000,216.0,5.000000,414.000000


In [29]:
# let's find the mean value of each process step 
data.groupby(['Workstation']).mean()

,ProcessID,RoundId,Seconds
Workstation,,,
1,1753.500000,216.0,62.764706
2,1753.117647,216.0,67.823529
3,1752.777778,216.0,98.611111
4,1751.520000,216.0,98.160000
5,1752.947368,216.0,41.052632


## Task: 
Please add the Produktionsauftrag.csv to the data and match the ProcessID with a variant. 

can you group the data based on the type of car and the workstation used? 

## Transformation / Feature engineering
Cleaning up data is part of nearly every machine learning project. ...


## End of Lab 1
We now have gained an understanding of our data and prepared our data ...